<a href="https://colab.research.google.com/github/VXLAKS/DZ3/blob/main/DZ3_KotovAS_%22Environment_example_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [85]:
!pip install -q -U kaggle_environments
!pip install numpy
!pip install pandas
!pip install matplotlib
!pip install seaborn


In [ ]:
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt
import seaborn as sns
from kaggle_environments import make, evaluate

print("Все библиотеки успешно импортированы!")


Все библиотеки успешно импортированы!


Опишем поведение агента, всегда играющего "камень" - это значение 0

In [ ]:
%%writefile rock_agent.py

#Example of the simple agent
#0 - rock
#1 - paper
#2 - scissors
def your_agent(observation, configuration):
    return 0

Overwriting rock_agent.py


Попробуем теперь использовать информацию о прошлых действиях противника. Опишем агента, который производит то же самое действие, что и оппонент на прошлом ходу

In [ ]:
%%writefile copy_opponent.py

#Example
def copy_opponent(observation, configuration):
    #in case we have information about opponent last move
    if observation.step > 0:
        return observation.lastOpponentAction
    #initial step
    else:
        return random.randrange(0, configuration.signs)

Overwriting copy_opponent.py


Воспользуемся функцией evaluate из библиотеки kaggle_environments с помощью которой запустим наших агентов и проведем эксперимент на заданном количестве игр

In [ ]:
evaluate(
    "rps", #environment to use - no need to change
    ["rock_agent.py", "copy_opponent.py"], #agents to evaluate
    configuration={"episodeSteps": 100} #number of episodes
)

[[1, None]]

In [ ]:
evaluate(
    "rps", #environment to use - no need to change
    ["rock_agent.py", "paper"], #agents to evaluate
    configuration={"episodeSteps": 100} #number of episodes
)

[[-99.0, 99.0]]

In [ ]:
%%writefile random_agent.py
def random_agent(observation, configuration):
    action = random.randrange(0, 3)
    return action


Overwriting random_agent.py


In [ ]:
%%writefile cyclic_agent.py
def cyclic_agent(observation, configuration):
    action = observation.step % 3
    return action


Overwriting cyclic_agent.py


In [ ]:
%%writefile always_paper_agent.py
def always_paper_agent(observation, configuration):
    return 1  # 1 означает "бумагу"


Writing always_paper_agent.py


In [ ]:
%%writefile always_scissors_agent.py

def always_scissors_agent(observation, configuration):
    return 2  # 2 означает "ножницы"


Writing always_scissors_agent.py


In [ ]:
%%writefile mirror_agent.py
def mirror_agent(observation, configuration):
    if observation.step == 0:
        return 0  # Начинает с "камня"
    else:
        return observation.lastOpponentAction  # Копирует предыдущее действие оппонента


Writing mirror_agent.py


In [88]:
%%writefile counter_agent.py
def counter_agent(observation, configuration):
    if observation.step == 0:
        return 0  # Начинает с "камня"
    else:
        return (observation.lastOpponentAction + 1) % 3  # Выбирает действие, побеждающее предыдущее действие оппонента


Overwriting counter_agent.py


In [87]:
%%writefile biased_random_agent.py
import random

def biased_random_agent(observation, configuration):
    probabilities = [0.5, 0.3, 0.2]  # Вероятности для "камень", "бумага", "ножницы"
    return random.choices([0, 1, 2], probabilities)[0]


Writing biased_random_agent.py


In [86]:
def evaluate(game, agents, configuration):
    scores = [0, 0]

    for step in range(configuration["episodeSteps"]):
        obs = type('obj', (object,), {'step': step})

        actions = [agent(obs, configuration) for agent in agents]  # Получение действий ботов

        # Определение победителя
        if actions[0] == (actions[1] + 1) % 3:
            scores[0] += 1
        elif actions[1] == (actions[0] + 1) % 3:
            scores[1] += 1


    return [[float(scores[0]), float(scores[1])]]

tournament_results = evaluate(
    "rps",
    [random_agent, cyclic_agent],
    configuration={"episodeSteps": 100}
)

print(tournament_results)


[[31.0, 33.0]]
